# Setup

In [60]:
import sklearn
assert sklearn.__version__ >= "0.20"

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Load dataset into variable

In [67]:
import pandas as pd

def load_asteroid_data():
    #csv_path = "shuffled_asteroids_head.csv"
    csv_path = "../dataset.csv"
    return pd.read_csv(csv_path, low_memory=False)

Features we can get rid of name, full_name, prefix, spkid, pdes. Split ID out and keep in seperate vector.

In [68]:
asteroids = load_asteroid_data()
#asteroids.head()
asteroids = asteroids.dropna(subset=['pha']) 
asteroids.info()

<class 'pandas.core.frame.DataFrame'>
Index: 938603 entries, 0 to 958523
Data columns (total 45 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              938603 non-null  object 
 1   spkid           938603 non-null  int64  
 2   full_name       938603 non-null  object 
 3   pdes            938603 non-null  object 
 4   name            22064 non-null   object 
 5   prefix          18 non-null      object 
 6   neo             938599 non-null  object 
 7   pha             938603 non-null  object 
 8   H               932341 non-null  float64
 9   diameter        136209 non-null  float64
 10  albedo          135103 non-null  float64
 11  diameter_sigma  136081 non-null  float64
 12  orbit_id        938603 non-null  object 
 13  epoch           938603 non-null  float64
 14  epoch_mjd       938603 non-null  int64  
 15  epoch_cal       938603 non-null  float64
 16  equinox         938603 non-null  object 
 17  e              

# Converting object datatypes to numerical and standardizing data

In [56]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
ordinal_encoder = OrdinalEncoder()

#dropping features we dont need
asteroids = asteroids.drop(columns=['spkid', 'full_name', 'pdes', 'prefix', 'name', 'orbit_id', 'equinox'])

#splitting features into numerical and categorical
asteroids_num = asteroids.drop(columns=["id", "neo", "pha", "class"])
asteroids_id = asteroids[["id"]]
asteroids_cat = asteroids[["neo", "pha", "class"]]

#encode categorical features
asteroid_cat_encoded = ordinal_encoder.fit_transform(asteroids_cat)

#Normalizing numerical features
std_scaler = StandardScaler()
asteroids_num_scaled = std_scaler.fit_transform(asteroids_num)



print(asteroid_cat_encoded)
print(asteroids_num_scaled)
    

[[ 0.  0.  6.]
 [ 0.  0.  6.]
 [ 0. nan  8.]
 ...
 [ 0.  0.  6.]
 [ 0.  0.  6.]
 [ 1.  0.  1.]]
[[-1.94986177 -0.03972371  1.66879174 ... -0.01227037 -0.01534474
  -1.21548345]
 [ 0.572599           nan         nan ... -0.01227037 -0.01534474
  -0.19090077]
 [-0.1726735          nan         nan ...         nan         nan
  -1.68393761]
 ...
 [ 0.572599           nan         nan ... -0.01227037 -0.01534474
  -0.0539467 ]
 [-0.57397408         nan         nan ... -0.01227037 -0.01534474
  -0.01070326]
 [ 5.73217784         nan         nan ... -0.01227036 -0.01533424
  -0.65104269]]
